In [258]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.impute import KNNImputer
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from xgboost import XGBClassifier
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e26/sample_submission.csv
/kaggle/input/playground-series-s3e26/train.csv
/kaggle/input/playground-series-s3e26/test.csv
/kaggle/input/cirrhosis-patient-survival-prediction/cirrhosis.csv


In [259]:
train_df = pd.read_csv('/kaggle/input/playground-series-s3e26/train.csv').drop(columns=['id'],axis=1)
test_df = pd.read_csv('/kaggle/input/playground-series-s3e26/test.csv')
submission_df = pd.read_csv('/kaggle/input/playground-series-s3e26/sample_submission.csv')
other_df = pd.read_csv('/kaggle/input/cirrhosis-patient-survival-prediction/cirrhosis.csv')[train_df.columns]

#train_df = pd.concat([train_df, other_df], ignore_index=True)
train_df = pd.concat(objs=[train_df, other_df]).reset_index(drop=True)
#train_df.drop(columns='ID', inplace=True)



# Combine imputed numerical and categorical data



train_df.dropna(inplace=True)
test_ids = test_df['id']
test_df.drop(columns=['id'], axis=1, inplace=True)
train_df

,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,999,D-penicillamine,21532,M,N,N,N,N,2.3,316.0,3.35,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,D
1,2574,Placebo,19237,F,N,N,N,N,0.9,364.0,3.54,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,C
2,3428,Placebo,13727,F,N,Y,Y,Y,3.3,299.0,3.55,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,D
3,2576,Placebo,18460,F,N,N,N,N,0.6,256.0,3.50,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,C
4,788,Placebo,16658,F,N,Y,N,N,1.1,346.0,3.65,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8212,1153,D-penicillamine,22347,F,N,Y,N,N,0.4,246.0,3.58,24.0,797.0,91.00,113.0,288.0,10.4,2.0,C
8213,994,Placebo,21294,F,N,N,N,N,0.4,260.0,2.75,41.0,1166.0,70.00,82.0,231.0,10.8,2.0,C
8214,939,D-penicillamine,22767,F,N,N,N,N,1.7,434.0,3.35,39.0,1713.0,171.00,100.0,234.0,10.2,2.0,C
8215,839,D-penicillamine,13879,F,N,N,N,N,2.0,247.0,3.16,69.0,1050.0,117.00,88.0,335.0,10.5,2.0,C


In [260]:
train_df['Drug'] = np.where(train_df['Drug'] == 'Placebo', 0, 1)
train_df['Sex'] = np.where(train_df['Sex'] == 'F', 0, 1)
train_df['Ascites'] = np.where(train_df['Ascites'] == 'N', 0, 1)
train_df['Hepatomegaly'] = np.where(train_df['Hepatomegaly'] == 'N', 0, 1)
train_df['Spiders'] = np.where(train_df['Spiders'] == 'N', 0, 1)

test_df['Drug'] = np.where(test_df['Drug'] == 'Placebo', 0, 1)
test_df['Sex'] = np.where(test_df['Sex'] == 'F', 0, 1)
test_df['Ascites'] = np.where(test_df['Ascites'] == 'N', 0, 1)
test_df['Hepatomegaly'] = np.where(test_df['Hepatomegaly'] == 'N', 0, 1)
test_df['Spiders'] = np.where(test_df['Spiders'] == 'N', 0, 1)

train_df = pd.get_dummies(train_df, columns=['Edema', 'Stage'], dtype=int,)
test_df = pd.get_dummies(test_df, columns=['Edema', 'Stage'], dtype=int,)

encoder = LabelEncoder()
train_df['Status'] = encoder.fit_transform(train_df['Status'])

train_df.columns

Index(['N_Days', 'Drug', 'Age', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders',
       'Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 'SGOT',
       'Tryglicerides', 'Platelets', 'Prothrombin', 'Status', 'Edema_N',
       'Edema_S', 'Edema_Y', 'Stage_1.0', 'Stage_2.0', 'Stage_3.0',
       'Stage_4.0'],
      dtype='object')

In [261]:
train_df["Age"] = (train_df['Age'] / 365.25).astype('float16')
test_df["Age"] = (test_df['Age'] / 365.25).astype('float16')
#train_df['Age_Group'] = pd.cut(train_df['Age'], bins=[19, 29, 49, 64, 99], labels = [0, 1, 2, 3]).astype('int16')
#train_df['Bilirubin_Albumin'] = train_df['Bilirubin'] * train_df['Albumin']

#train_df.drop(columns=['Age'], inplace=True)

train_df['Bilirubin_in_norm'] = np.where(train_df['Bilirubin'] < 1.2, 1,0)
train_df['Cholesterol_in_norm'] = np.where(train_df['Cholesterol'] < 200, 1,0)
train_df['Albumin_in_norm'] = np.where((train_df['Albumin'] < 5.4) & (train_df['Albumin'] > 3.4), 1,0)
train_df['Copper_in_norm'] = np.where((train_df['Copper'] < 30) & (train_df['Copper'] > 10), 1,0)
train_df['Alk_Phos_in_norm'] = np.where((train_df['Alk_Phos'] < 129) & (train_df['Alk_Phos'] > 40), 1,0)
train_df['SGOT_in_norm'] = np.where((train_df['SGOT'] < 45) & (train_df['SGOT'] > 8), 1,0)
train_df['Tryglicerides_in_norm'] = np.where((train_df['Tryglicerides'] < 150) & (train_df['Tryglicerides'] > 0), 1,0)
#train_df['Platelets_in_norm'] = np.where((train_df['Platelets'] < 400) & (train_df['Platelets'] > 150), 1,0)
#train_df['Prothrombin_in_norm'] = np.where((train_df['Prothrombin'] < 12.5) & (train_df['Prothrombin'] > 9.4), 1,0)
test_df['Bilirubin_in_norm'] = np.where(test_df['Bilirubin'] < 1.2, 1,0)
test_df['Cholesterol_in_norm'] = np.where(test_df['Cholesterol'] < 200, 1,0)
test_df['Albumin_in_norm'] = np.where((test_df['Albumin'] < 5.4) & (test_df['Albumin'] > 3.4), 1,0)
test_df['Copper_in_norm'] = np.where((test_df['Copper'] < 30) & (test_df['Copper'] > 10), 1,0)
test_df['Alk_Phos_in_norm'] = np.where((test_df['Alk_Phos'] < 129) & (test_df['Alk_Phos'] > 40), 1,0)
test_df['SGOT_in_norm'] = np.where((test_df['SGOT'] < 45) & (test_df['SGOT'] > 8), 1,0)
test_df['Tryglicerides_in_norm'] = np.where((test_df['Tryglicerides'] < 150) & (test_df['Tryglicerides'] > 0), 1,0)

#symptom_columns = ['Ascites', 'Hepatomegaly', 'Spiders']
#train_df['Symptom_Score'] = train_df[symptom_columns].sum(axis=1)
#test_df['Symptom_Score'] = test_df[symptom_columns].sum(axis=1)

num_cols = ['N_Days', 'Age', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin' ]

for col in num_cols:
    lower = np.quantile(train_df[col], 0.25)
    upper = np.quantile(train_df[col], 0.75)
    iqr = upper - lower
    train_df_outliers = train_df[(train_df[col] > lower - 1.5*iqr) & (train_df[col] < upper + 1.5*iqr)]

print(train_df.shape, train_df_outliers.shape)
train_df = train_df_outliers.copy()

In [262]:
X_train = train_df.drop(columns='Status')
y_train = train_df['Status']



kmeans = KMeans(n_clusters=3, random_state=5, n_init=10).fit(X_train)

test_clusters = kmeans.predict(test_df)
X_train['cluster_labels'] = kmeans.labels_
test_df['cluster_labels'] = test_clusters



from sklearn.preprocessing import RobustScaler

sc = RobustScaler()

X_train = pd.DataFrame(sc.fit_transform(X_train), columns=X_train.columns)
test_df = pd.DataFrame(sc.transform(test_df), columns=test_df.columns)

from imblearn.under_sampling import TomekLinks

tl = TomekLinks(sampling_strategy='auto')
X_train , y_train = tl.fit_resample(X_train, y_train)

CatBoost - Best parameters: {'learning_rate': 0.09961907322421582, 'depth': 4, 'l2_leaf_reg': 4.024740752083527, 'border_count': 252, 'loss_function': 'MultiClass', 'eval_metric': 'MultiClass'}

LGBM PARAMS = Best parameters: {'learning_rate': 0.09267708260918013, 'num_leaves': 598, 'max_depth': 9, 'min_child_samples': 66, 'subsample': 0.8465101335020218, 'colsample_bytree': 0.7094458449756161, 'reg_alpha': 0.009277691053032699, 'reg_lambda': 0.0011673378745859116, 'objective': 'multiclass', 'num_class': 3}

In [263]:
lgbm_params = {'learning_rate': 0.09267708260918013, 'num_leaves': 598, 'max_depth': 9, 'min_child_samples': 66,
               'subsample': 0.8465101335020218, 'colsample_bytree': 0.7094458449756161, 'reg_alpha': 0.009277691053032699,
               'reg_lambda': 0.0011673378745859116, 'objective': 'multiclass', 'num_class': 3}

cat_params = {'learning_rate': 0.09961907322421582, 'depth': 4, 'l2_leaf_reg': 4.024740752083527, 'border_count': 252,
              'loss_function': 'MultiClass', 'eval_metric': 'MultiClass'}

xgb_params = {'eta': 0.28467782664138885, 'max_depth': 7, 'subsample': 0.8547528315009247,
              'colsample_bytree': 0.5463751470032918, 'alpha': 0.00524431563710506, 'lambda': 0.03455006552734786,
              'objective': 'multi:softprob', 'eval_metric': 'mlogloss'}

In [264]:
kfold = StratifiedKFold(n_splits=5, random_state=5, shuffle=True)

results_xgb = cross_val_score(XGBClassifier(objective='multi_logloss', n_jobs=-1, random_state=5), X_train , y_train, 
                          cv=kfold, n_jobs=-1, scoring='neg_log_loss',) * -1
print(np.mean(results_xgb))



In [265]:
lgbm_clf = lgb.LGBMClassifier(**lgbm_params)
catboost_clf = CatBoostClassifier(**cat_params, verbose=0)
xgb_model = XGBClassifier(**xgb_params)

voting_clf = VotingClassifier(
    estimators=[
        ('lgbm', lgbm_clf),
        ('catboost', catboost_clf),
        ('xgb', xgb_model)
    ],
    voting='soft',
    n_jobs=-1
)
#results_vot = cross_val_score(voting_clf, X_train , y_train, 
                          #cv=kfold, n_jobs=-1, scoring='neg_log_loss',) * -1
#print(np.mean(results_vot))


Voter - 0.444

In [266]:
voting_clf.fit(X_train, y_train)

results = voting_clf.predict_proba(test_df)
results_C = results[:,0]
results_CL = results[:, 1]
results_D = results[:, 2]

data = {
    'id': test_ids,
    'Status_C': results_C,
    'Status_CL': results_CL,
    'Status_D': results_D
}

submission_df_my = pd.DataFrame(data)


In [267]:

submission_df_my.to_csv('/kaggle/working/submission_4.csv', index=False)

test = pd.read_csv('/kaggle/working/submission_4.csv')
#test.drop(columns='Unnamed: 0', inplace=True)
test.head()

,id,Status_C,Status_CL,Status_D
0,7905,0.395807,0.018270,0.585922
1,7906,0.569687,0.123928,0.306385
2,7907,0.023592,0.007123,0.969285
3,7908,0.964701,0.002104,0.033195
4,7909,0.924542,0.017454,0.058004


RFC - 0.521
XGB 